## GAN

In [13]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.initializers import random_normal

import matplotlib.pyplot as plt

import sys

import numpy as np

In [15]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, kernel_initializer=random_normal(), input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(1024, kernel_initializer=random_normal()))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones(batch_size)
        fake = np.zeros(batch_size)

        for epoch in range(epochs+1):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [19]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=1000, batch_size=2000, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 256)               0         
__________

C:\Users\Jwp\Anaconda3\envs\py_ml\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.491985, acc.: 56.90%] [G loss: 0.873986]
2 [D loss: 0.251037, acc.: 94.05%] [G loss: 1.528532]
3 [D loss: 0.123169, acc.: 99.95%] [G loss: 2.120720]
4 [D loss: 0.075529, acc.: 100.00%] [G loss: 2.548415]
5 [D loss: 0.057986, acc.: 100.00%] [G loss: 2.794393]
6 [D loss: 0.050469, acc.: 100.00%] [G loss: 2.959202]
7 [D loss: 0.042778, acc.: 100.00%] [G loss: 3.109859]
8 [D loss: 0.037745, acc.: 100.00%] [G loss: 3.240162]
9 [D loss: 0.033494, acc.: 100.00%] [G loss: 3.367568]
10 [D loss: 0.029751, acc.: 100.00%] [G loss: 3.520442]
11 [D loss: 0.024936, acc.: 100.00%] [G loss: 3.629642]
12 [D loss: 0.022724, acc.: 100.00%] [G loss: 3.767442]
13 [D loss: 0.021193, acc.: 99.98%] [G loss: 3.858801]
14 [D loss: 0.018754, acc.: 100.00%] [G loss: 4.021739]
15 [D loss: 0.016756, acc.: 100.00%] [G loss: 4.117403]
16 [D loss: 0.015267, acc.: 100.00%] [G loss: 4.206571]
17 [D loss: 0.014107, acc.: 100.00%] [G loss: 4.308062]
18 [D loss: 0.013274, acc.: 100.00%] [G loss: 4.394947]
19 [D

149 [D loss: 0.190040, acc.: 98.57%] [G loss: 3.393847]
150 [D loss: 0.266404, acc.: 94.92%] [G loss: 2.471030]
151 [D loss: 0.220061, acc.: 90.40%] [G loss: 4.060589]
152 [D loss: 0.461097, acc.: 70.23%] [G loss: 2.149742]
153 [D loss: 0.212699, acc.: 93.62%] [G loss: 3.702365]
154 [D loss: 0.494626, acc.: 68.15%] [G loss: 2.503268]
155 [D loss: 0.187417, acc.: 96.60%] [G loss: 4.011427]
156 [D loss: 0.437004, acc.: 77.80%] [G loss: 2.126423]
157 [D loss: 0.183736, acc.: 98.58%] [G loss: 3.026796]
158 [D loss: 0.466835, acc.: 77.40%] [G loss: 2.108689]
159 [D loss: 0.191129, acc.: 92.55%] [G loss: 3.614312]
160 [D loss: 0.355060, acc.: 86.52%] [G loss: 2.259910]
161 [D loss: 0.203701, acc.: 94.50%] [G loss: 3.192406]
162 [D loss: 0.437138, acc.: 82.80%] [G loss: 2.365353]
163 [D loss: 0.211935, acc.: 97.40%] [G loss: 3.292119]
164 [D loss: 0.625419, acc.: 56.27%] [G loss: 1.462229]
165 [D loss: 0.282530, acc.: 85.55%] [G loss: 2.567493]
166 [D loss: 0.556791, acc.: 71.25%] [G loss: 1.

296 [D loss: 0.380384, acc.: 91.08%] [G loss: 2.408205]
297 [D loss: 0.367747, acc.: 92.47%] [G loss: 2.409122]
298 [D loss: 0.358851, acc.: 92.75%] [G loss: 2.365608]
299 [D loss: 0.320276, acc.: 92.82%] [G loss: 2.502269]
300 [D loss: 0.422457, acc.: 82.75%] [G loss: 2.402270]
301 [D loss: 0.339301, acc.: 93.97%] [G loss: 2.430010]
302 [D loss: 0.424968, acc.: 82.55%] [G loss: 2.643433]
303 [D loss: 0.451811, acc.: 84.42%] [G loss: 2.559812]
304 [D loss: 0.349117, acc.: 93.38%] [G loss: 2.750500]
305 [D loss: 0.478903, acc.: 79.93%] [G loss: 2.675185]
306 [D loss: 0.352869, acc.: 91.97%] [G loss: 2.654863]
307 [D loss: 0.419928, acc.: 82.20%] [G loss: 2.833403]
308 [D loss: 0.493369, acc.: 77.10%] [G loss: 2.605088]
309 [D loss: 0.338634, acc.: 92.65%] [G loss: 2.694551]
310 [D loss: 0.452772, acc.: 78.08%] [G loss: 2.732785]
311 [D loss: 0.435133, acc.: 84.48%] [G loss: 2.604622]
312 [D loss: 0.413247, acc.: 85.53%] [G loss: 2.730121]
313 [D loss: 0.446738, acc.: 83.60%] [G loss: 2.

443 [D loss: 0.467521, acc.: 75.87%] [G loss: 3.429707]
444 [D loss: 0.265076, acc.: 89.82%] [G loss: 3.991855]
445 [D loss: 0.459208, acc.: 77.65%] [G loss: 3.398256]
446 [D loss: 0.250025, acc.: 90.08%] [G loss: 3.860189]
447 [D loss: 0.383301, acc.: 84.05%] [G loss: 3.305679]
448 [D loss: 0.231533, acc.: 96.77%] [G loss: 3.524150]
449 [D loss: 0.353233, acc.: 89.75%] [G loss: 3.654751]
450 [D loss: 0.349029, acc.: 89.23%] [G loss: 3.438555]
451 [D loss: 0.240049, acc.: 95.88%] [G loss: 3.553129]
452 [D loss: 0.316105, acc.: 91.60%] [G loss: 3.810696]
453 [D loss: 0.379937, acc.: 88.52%] [G loss: 3.680097]
454 [D loss: 0.247753, acc.: 93.97%] [G loss: 3.261126]
455 [D loss: 0.235100, acc.: 96.80%] [G loss: 3.682425]
456 [D loss: 0.298512, acc.: 93.47%] [G loss: 3.678545]
457 [D loss: 0.236282, acc.: 96.05%] [G loss: 3.507010]
458 [D loss: 0.243562, acc.: 96.38%] [G loss: 3.838997]
459 [D loss: 0.310461, acc.: 93.30%] [G loss: 3.804195]
460 [D loss: 0.208936, acc.: 95.82%] [G loss: 3.

590 [D loss: 0.280635, acc.: 91.67%] [G loss: 4.489670]
591 [D loss: 0.159165, acc.: 96.42%] [G loss: 4.423234]
592 [D loss: 0.249706, acc.: 92.68%] [G loss: 5.740496]
593 [D loss: 0.686699, acc.: 61.45%] [G loss: 6.876118]
594 [D loss: 0.337860, acc.: 84.60%] [G loss: 3.758067]
595 [D loss: 0.151023, acc.: 97.48%] [G loss: 4.815889]
596 [D loss: 0.086018, acc.: 97.70%] [G loss: 4.394578]
597 [D loss: 0.233358, acc.: 93.55%] [G loss: 4.613567]
598 [D loss: 0.215868, acc.: 94.02%] [G loss: 4.175692]
599 [D loss: 0.201100, acc.: 93.62%] [G loss: 4.565181]
600 [D loss: 0.286440, acc.: 91.03%] [G loss: 4.526464]
601 [D loss: 0.186832, acc.: 92.10%] [G loss: 5.135504]
602 [D loss: 0.348260, acc.: 84.38%] [G loss: 4.763655]
603 [D loss: 0.246544, acc.: 88.57%] [G loss: 5.745213]
604 [D loss: 0.494741, acc.: 75.30%] [G loss: 5.083505]
605 [D loss: 0.143713, acc.: 95.12%] [G loss: 5.257413]
606 [D loss: 0.168558, acc.: 95.72%] [G loss: 3.823623]
607 [D loss: 0.170845, acc.: 98.25%] [G loss: 4.

737 [D loss: 0.230416, acc.: 94.88%] [G loss: 4.937872]
738 [D loss: 0.228096, acc.: 92.15%] [G loss: 4.776790]
739 [D loss: 0.174234, acc.: 94.85%] [G loss: 4.965078]
740 [D loss: 0.178655, acc.: 96.03%] [G loss: 4.454633]
741 [D loss: 0.153735, acc.: 97.05%] [G loss: 4.620976]
742 [D loss: 0.194305, acc.: 95.88%] [G loss: 4.885291]
743 [D loss: 0.199662, acc.: 95.00%] [G loss: 4.688268]
744 [D loss: 0.168297, acc.: 95.53%] [G loss: 5.342450]
745 [D loss: 0.316814, acc.: 84.65%] [G loss: 6.261930]
746 [D loss: 0.386735, acc.: 80.10%] [G loss: 6.276486]
747 [D loss: 0.296846, acc.: 84.60%] [G loss: 6.717633]
748 [D loss: 0.201008, acc.: 93.12%] [G loss: 3.961366]
749 [D loss: 0.162254, acc.: 94.80%] [G loss: 6.227624]
750 [D loss: 0.353165, acc.: 82.57%] [G loss: 5.439435]
751 [D loss: 0.176916, acc.: 93.75%] [G loss: 4.504494]
752 [D loss: 0.121360, acc.: 98.15%] [G loss: 4.448655]
753 [D loss: 0.140057, acc.: 97.40%] [G loss: 4.630520]
754 [D loss: 0.208039, acc.: 96.13%] [G loss: 5.

884 [D loss: 0.217006, acc.: 92.77%] [G loss: 4.590529]
885 [D loss: 0.186317, acc.: 92.15%] [G loss: 6.111562]
886 [D loss: 0.442271, acc.: 75.40%] [G loss: 6.045447]
887 [D loss: 0.185694, acc.: 91.18%] [G loss: 6.877862]
888 [D loss: 0.132312, acc.: 94.92%] [G loss: 4.295366]
889 [D loss: 0.123280, acc.: 97.10%] [G loss: 5.142423]
890 [D loss: 0.089359, acc.: 97.12%] [G loss: 4.303138]
891 [D loss: 0.151434, acc.: 95.65%] [G loss: 5.727878]
892 [D loss: 0.255606, acc.: 91.40%] [G loss: 4.619656]
893 [D loss: 0.094121, acc.: 98.05%] [G loss: 5.146233]
894 [D loss: 0.105968, acc.: 97.30%] [G loss: 4.184152]
895 [D loss: 0.124171, acc.: 98.25%] [G loss: 5.355712]
896 [D loss: 0.177662, acc.: 95.23%] [G loss: 4.520927]
897 [D loss: 0.104574, acc.: 98.10%] [G loss: 4.694378]
898 [D loss: 0.139025, acc.: 96.95%] [G loss: 5.466395]
899 [D loss: 0.320407, acc.: 89.15%] [G loss: 6.942307]
900 [D loss: 0.224013, acc.: 91.75%] [G loss: 3.783622]
901 [D loss: 0.098474, acc.: 98.98%] [G loss: 5.

In [18]:
60000/128

468.75